In [1]:
import pandas as pd
import json
import numpy as np
from types import NoneType
import re
from IPython.display import display, Markdown

# !pip install mlxtend
# from mlxtend.preprocessing import TransactionEncoder
# from mlxtend.frequent_patterns import apriori, association_rules
import ast
import json
from collections import defaultdict
from itertools import combinations, chain, product

from google import genai
from google.genai import types
import os
os.chdir("/Users/vophuhan/Everything/University/Năm 4/Semester 2/Ứng dụng Phân tích dữ liệu thông minh/21KHDL-TikTok-Analytics")

In [2]:
features_df = pd.read_parquet(
    'data/content-features-dataset/content_features-new-with-stats-top50WeeklyVideos.parquet')
# features_df = pd.read_csv('content_features-raw-top50WeeklyVideos.csv')

features_df = features_df[features_df['categories']
                          != 'Không liên quan ẩm thực']

In [3]:
features_df['year'] = features_df['createTime'].dt.isocalendar().year
features_df['week'] = features_df['createTime'].dt.isocalendar().week

In [ ]:
features_df['engagement_rate'] = (features_df['statsV2.diggCount'] + features_df['statsV2.commentCount'] +
                                  features_df['statsV2.shareCount'] + features_df['statsV2.collectCount']) / features_df['statsV2.playCount']

In [ ]:
features_df['transcript_len'] = features_df['transcript'].apply(len)
features_df['cta_count'] = features_df['cta_content'].apply(len)
features_df = features_df[features_df['transcript_len'] > 0]

features_df['transcript_word_count'] = features_df['transcript'].str.split().str.len()

In [ ]:
mean_word_per_second = features_df['transcript_word_count'].mean(
) / features_df['video.duration'].mean()

features_df.loc[features_df['video.duration'] == 0,
                'video.duration'] = features_df['transcript_word_count'] / mean_word_per_second
features_df['word_per_second'] = features_df['transcript_word_count'] / \
    features_df['video.duration']

features_df = features_df[features_df['word_per_second'] < 8]

In [ ]:
def parse_list(x):
    if isinstance(x, np.ndarray):
        return list(x)
    if isinstance(x, NoneType):
        return []


for col in features_df.columns:
    str_flag = features_df[col].apply(
        lambda x: isinstance(x, np.ndarray)).any()
    if str_flag:
        features_df[col] = features_df[col].apply(parse_list)

In [ ]:
features_df.to_parquet(
    'data/ScriptGen-data/script_gendata.parquet', index=False)

In [9]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2652 entries, 0 to 3003
Data columns (total 43 columns):
 #   Column                      Non-Null Count  Dtype                           
---  ------                      --------------  -----                           
 0   categories                  2652 non-null   object                          
 1   audience_target             2652 non-null   object                          
 2   body_analysis               2652 non-null   object                          
 3   content_style               2652 non-null   object                          
 4   cta_content                 2652 non-null   object                          
 5   cta_type                    2652 non-null   object                          
 6   emotion_keywords            2652 non-null   object                          
 7   has_cta                     2652 non-null   bool                            
 8   highlight_sentences         2652 non-null   object                       

In [3]:
df = pd.read_parquet('data/ScriptGen-data/script_gendata.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2652 entries, 0 to 2651
Data columns (total 43 columns):
 #   Column                      Non-Null Count  Dtype                           
---  ------                      --------------  -----                           
 0   categories                  2652 non-null   object                          
 1   audience_target             2652 non-null   object                          
 2   body_analysis               2652 non-null   object                          
 3   content_style               2652 non-null   object                          
 4   cta_content                 2652 non-null   object                          
 5   cta_type                    2652 non-null   object                          
 6   emotion_keywords            2652 non-null   object                          
 7   has_cta                     2652 non-null   bool                            
 8   highlight_sentences         2652 non-null   object                  

In [ ]:
def remove_hashtags(text):
    return re.sub(r'\s*#\w+', '', text).strip()


# Tạo cột mới: desc không có hashtag
df['desc_clean'] = df['desc'].apply(remove_hashtags)

# Tạo cột đếm số từ trong desc_clean
df['desc_word_count'] = df['desc_clean'].apply(
    lambda x: len(x.split()) if isinstance(x, str) else 0)

In [7]:
df.to_parquet(
    'data/ScriptGen-data/script_gendata.parquet', index=False)